In [1]:
# import modules
import pandas as pd 
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
import geopandas as gpd
from rasterstats import zonal_stats
from osgeo import gdal
from tqdm.notebook import tqdm
import multiprocessing
from functools import partial
import math
# import calcArea from separate script, won't run when defined in this script
import calc_area 

# track time
import time
start = time.time()
last = time.time()

### calculate area (sqkm) for given slope & distance thresholds

In [2]:
### define functions ###
# function to generate buffers for each FUA
def genBuffers(gdf, distance_thresholds):
    # import thresholds and select radii 
    distances = pd.read_csv(distance_thresholds)
    radii = distances.radii.dropna()
    print(radii)
    # buffer each FUA by distance thresholds and add to fua_gdf
    for value in radii:
        # convert value to metres
        value_m = value * 1000 
        gdf[str(int(value)) + 'km'] = gdf.buffer(value_m)
    print(gdf)
    
    return gdf

# define function to iterate over each row in fua_gdf calculating area values using multiprocessing
def process_developable_land_mp(gdf):
    developable_land_dict = {}
    for row in tqdm(gdf.itertuples(), total=len(gdf)):
        aoi = row[1]
        distances = row[-14:]
        img = row[6] 
        pool = multiprocessing.Pool(processes=4)
        # calcArea process imported from separate script calc_area.py to run in jupyter
        run_calc_area = partial(calc_area.calcArea, img=img)
        results = pool.map(run_calc_area, distances)
        # convert results to dict
        results_dict = dict((key, val) for k in results for key, val in k.items())
        developable_land_dict[aoi] = results_dict

    return developable_land_dict



### run

In [3]:
if __name__ == '__main__':
    # read FUAs as geodataframe reproject to NZTM (EPSG:2193)
    fua_df = pd.read_excel('inputs/FUA_coordinates.xlsx')
    fua_gdf = gpd.GeoDataFrame(fua_df, geometry=gpd.points_from_xy(fua_df.long, fua_df.lat), crs='EPSG:4326').to_crs(2193)

    # add image file path as column in fua_gdf
    fua_gdf['imgPath'] = 'outputs/slope_images/' + fua_gdf.FUA + '-slope.kea'

    # generate buffers
    fua_gdf = genBuffers(fua_gdf, 'inputs/thresholds.csv')

    print('distance thresholds generated')
    print(time.time() - last, 'seconds')
    last = time.time()
    print()

    output = process_developable_land_mp(fua_gdf)

    print('pixel sums generated')
    print(time.time() - last, 'seconds')
    last = time.time()
    print()

    # reshape dict to pd.Dataframe and write to csv
    output_df = pd.DataFrame.from_dict(output, orient='columns')
    output_df.to_csv('outputs/area-under-thresholds.csv')

0      5.0
1     10.0
2     15.0
3     20.0
4     25.0
      ... 
9     50.0
10    55.0
11    60.0
12    65.0
13    70.0
Name: radii, Length: 14, dtype: float64
             FUA        lat        long  \
0       Auckland -36.848460  174.763332   
1       Hamilton -37.787625  175.281219   
2       Tauranga -37.683042  176.168700   
3     Wellington -41.288832  174.776908   
4   Christchurch -43.530822  172.637056   
..           ...        ...         ...   
48       Motueka -41.107109  173.014602   
49      Cromwell -45.047977  169.193679   
50     Alexandra -45.256338  169.389510   
51        Wanaka -44.693538  169.138970   
52          Gore -46.101641  168.941620   

   City council offices. Otherwise regional council office unless otherwise stated  \
0                                            skytower                                
1                                                 NaN                                
2                                                 NaN           

  0%|          | 0/53 [00:00<?, ?it/s]

pixel sums generated
11624.342360019684 seconds

